In [1]:
# Word Count

In [2]:
import findspark
findspark.init()

In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName('word count')\
        .getOrCreate()
sc=spark.sparkContext

In [14]:
in_text=sc.textFile('/Users/macproretina132015/Desktop/text7205.txt')

In [15]:
in_text.collect()

['A nose print of a dog is unique the same way fingerprints of a human being are unique.',
 'A human nose has five million scent receptors, while  a dog nose has up to three hundred million receptors.']

In [16]:
text_flat=in_text.flatMap(lambda x: x.split(' '))\
            .map(lambda x: (x,1)) \
            .reduceByKey(lambda x,y :x+y)

In [17]:
text_flat.collect()

[('A', 2),
 ('nose', 3),
 ('print', 1),
 ('of', 2),
 ('a', 3),
 ('dog', 2),
 ('is', 1),
 ('unique', 1),
 ('the', 1),
 ('same', 1),
 ('way', 1),
 ('fingerprints', 1),
 ('human', 2),
 ('being', 1),
 ('are', 1),
 ('unique.', 1),
 ('has', 2),
 ('five', 1),
 ('million', 2),
 ('scent', 1),
 ('receptors,', 1),
 ('while', 1),
 ('', 1),
 ('up', 1),
 ('to', 1),
 ('three', 1),
 ('hundred', 1),
 ('receptors.', 1)]

In [12]:
sc.stop()
spark.stop()

In [9]:
# TF-IDF

In [138]:
text_2=in_text.flatMap(lambda x: x.split('.'))

In [139]:
a=text_2.collect()

In [142]:
for i in range(len(a)-1) :
    if len(a[i])== 0 :
        del a[i]
        

In [168]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [173]:
sentenceData = spark.createDataFrame([
    (0, a[0]),
    (1, a[1])],
 ["document", "sentence"])

In [174]:
sentenceData.show(truncate=False)

+--------+----------------------------------------------------------------------------------------------------------+
|document|sentence                                                                                                  |
+--------+----------------------------------------------------------------------------------------------------------+
|0       |A nose print of a dog is unique the same way fingerprints of a human being are unique                     |
|1       |A human nose has five million scent receptors, while  a dog nose has up to three hundred million receptors|
+--------+----------------------------------------------------------------------------------------------------------+



In [176]:
from pyspark.ml import Pipeline
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
vectorizer  = CountVectorizer(inputCol="words", outputCol="rawFeatures")

idf = IDF(inputCol="rawFeatures", outputCol="features")

pipeline = Pipeline(stages=[tokenizer, vectorizer, idf])

model = pipeline.fit(sentenceData)

In [177]:
import numpy as np

total_counts = model.transform(sentenceData)\
                    .select('rawFeatures').rdd\
                    .map(lambda row: row['rawFeatures'].toArray())\
                    .reduce(lambda x,y: [x[i]+y[i] for i in range(len(y))])

vocabList = model.stages[1].vocabulary
d = {'vocabList':vocabList,'counts':total_counts}

spark.createDataFrame(np.array(list(d.values())).T.tolist(),list(d.keys())).show()

+------------+------+
|   vocabList|counts|
+------------+------+
|           a|   5.0|
|        nose|   3.0|
|         has|   2.0|
|      unique|   2.0|
|       human|   2.0|
|         dog|   2.0|
|     million|   2.0|
|          of|   2.0|
|         are|   1.0|
|          up|   1.0|
|        same|   1.0|
|          to|   1.0|
|          is|   1.0|
|       scent|   1.0|
|       print|   1.0|
|     hundred|   1.0|
|fingerprints|   1.0|
|       three|   1.0|
|   receptors|   1.0|
|         way|   1.0|
+------------+------+
only showing top 20 rows



In [187]:
model.transform(sentenceData).show(truncate=False,vertical=True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 document    | 0                                                                                                                                                                                                                                                                                                     
 sentence    | A nose print of a dog is unique the same way fingerprints of a human being are unique                                                                                                                                                                                                                 
 words       | [a, nose, print, of, a, dog, is, unique, the, same, way